<a href="https://colab.research.google.com/github/aquimfrsilva/Trabalho_Boss_Final/blob/main/Verde_do_amanh%C3%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Sistema de Gerenciamento de Usuários - Verde do Amanhã
Versão simplificada com tratamento de dados básico
"""

import pandas as pd
from datetime import datetime
import numpy as np
import re

class GerenciadorUsuarios:
    def __init__(self):
        self.usuarios = pd.DataFrame({
            'ID': [1, 2, 3, 4, 5],
            'Nome': ['Ana Silva', 'Carlos Santos', 'Maria Oliveira', 'João Costa', 'Pedro Lima'],
            'Email': ['ana@verdeamanha.com', 'carlos@verdeamanha.com', 'maria@verdeamanha.com',
                     'joao@verdeamanha.com', 'pedro@cliente.com'],
            'Tipo': ['Admin', 'Gerente', 'Especialista', 'Tecnico', 'Cliente'],
            'Especialidade': [None, 'Eng. Ambiental', 'Biologia', 'Agronomia', None],
            'Status': ['Ativo', 'Ativo', 'Ativo', 'Ativo', 'Ativo'],
            'Data_Cadastro': ['2024-01-15', '2024-01-20', '2024-02-01', '2024-02-15', '2024-03-01']
        })

        # Converter data para datetime
        self.usuarios['Data_Cadastro'] = pd.to_datetime(self.usuarios['Data_Cadastro'])

        # Listas de valores válidos
        self.tipos_validos = ['Admin', 'Gerente', 'Especialista', 'Tecnico', 'Cliente']
        self.status_validos = ['Ativo', 'Inativo', 'Suspenso']

    def limpar_dados(self):
        """Remove espaços e padroniza formato dos dados"""
        self.usuarios['Nome'] = self.usuarios['Nome'].str.strip()
        self.usuarios['Email'] = self.usuarios['Email'].str.lower().str.strip()
        self.usuarios['Tipo'] = self.usuarios['Tipo'].str.title()

        # Remove duplicatas por email
        self.usuarios = self.usuarios.drop_duplicates(subset=['Email'], keep='first')
        print("Dados limpos com sucesso.")

    def validar_email(self, email):
        """Verifica se email tem formato válido"""
        padrao = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return re.match(padrao, email) is not None

    def adicionar_usuario(self, nome, email, tipo, especialidade=None):
        """Adiciona novo usuário com validações básicas"""
        # Validações
        if not nome or len(nome.strip()) < 2:
            return "Erro: Nome deve ter pelo menos 2 caracteres"

        if not self.validar_email(email):
            return "Erro: Email inválido"

        if email.lower() in self.usuarios['Email'].str.lower().values:
            return "Erro: Email já cadastrado"

        if tipo not in self.tipos_validos:
            return f"Erro: Tipo deve ser um dos: {', '.join(self.tipos_validos)}"

        # Criar novo usuário
        novo_id = self.usuarios['ID'].max() + 1 if not self.usuarios.empty else 1

        novo_usuario = pd.DataFrame({
            'ID': [novo_id],
            'Nome': [nome.strip().title()],
            'Email': [email.lower().strip()],
            'Tipo': [tipo],
            'Especialidade': [especialidade],
            'Status': ['Ativo'],
            'Data_Cadastro': [datetime.now()]
        })

        self.usuarios = pd.concat([self.usuarios, novo_usuario], ignore_index=True)
        return f"Usuario {nome} adicionado com sucesso. ID: {novo_id}"

    def filtrar_usuarios(self, campo=None, valor=None):
        """Filtra usuários por um campo específico"""
        if not campo or not valor:
            return self.usuarios

        if campo not in self.usuarios.columns:
            print(f"Campo '{campo}' não existe.")
            return pd.DataFrame()

        if campo in ['Nome', 'Email', 'Tipo', 'Especialidade', 'Status']:
            return self.usuarios[self.usuarios[campo].str.contains(valor, case=False, na=False)]
        else:
            return self.usuarios[self.usuarios[campo] == valor]

    def buscar_por_id(self, user_id):
        """Busca usuário por ID"""
        return self.usuarios[self.usuarios['ID'] == user_id]

    def atualizar_status(self, user_id, novo_status):
        """Atualiza status do usuário"""
        if novo_status not in self.status_validos:
            return f"Erro: Status deve ser um dos: {', '.join(self.status_validos)}"

        if user_id in self.usuarios['ID'].values:
            self.usuarios.loc[self.usuarios['ID'] == user_id, 'Status'] = novo_status
            nome = self.usuarios.loc[self.usuarios['ID'] == user_id, 'Nome'].iloc[0]
            return f"Status de {nome} atualizado para {novo_status}"
        else:
            return "Erro: Usuario não encontrado"

    def contar_por_tipo(self):
        """Conta usuários por tipo"""
        return self.usuarios['Tipo'].value_counts()

    def contar_por_status(self):
        """Conta usuários por status"""
        return self.usuarios['Status'].value_counts()

    def listar_especialidades(self):
        """Lista especialidades cadastradas"""
        return self.usuarios['Especialidade'].dropna().value_counts()

    def mostrar_todos(self):
        """Exibe todos os usuários"""
        df_display = self.usuarios.copy()
        df_display['Data_Cadastro'] = df_display['Data_Cadastro'].dt.strftime('%d/%m/%Y')
        print("\nLISTA DE USUARIOS CADASTRADOS")
        print("="*70)
        print(df_display.to_string(index=False))
        print(f"\nTotal: {len(self.usuarios)} usuarios")

    def exportar_csv(self, nome_arquivo='usuarios.csv'):
        """Exporta dados para CSV"""
        self.usuarios.to_csv(nome_arquivo, index=False, encoding='utf-8')
        return f"Dados exportados para {nome_arquivo}"

def menu_interativo():
    """Menu principal simplificado"""
    gerenciador = GerenciadorUsuarios()

    while True:
        print("\n" + "="*50)
        print("SISTEMA DE USUARIOS - Verde do Amanha")
        print("="*50)
        print("1. Listar todos os usuarios")
        print("2. Adicionar novo usuario")
        print("3. Buscar usuario por ID")
        print("4. Filtrar usuarios")
        print("5. Atualizar status")
        print("6. Relatorios")
        print("7. Exportar dados")
        print("8. Limpar dados")
        print("9. Sair")

        opcao = input("\nEscolha uma opcao: ").strip()

        if opcao == "1":
            gerenciador.mostrar_todos()

        elif opcao == "2":
            print("\nADICIONAR NOVO USUARIO")
            nome = input("Nome: ")
            email = input("Email: ")
            print(f"Tipos validos: {', '.join(gerenciador.tipos_validos)}")
            tipo = input("Tipo: ")
            especialidade = input("Especialidade (opcional): ")
            especialidade = especialidade if especialidade else None

            resultado = gerenciador.adicionar_usuario(nome, email, tipo, especialidade)
            print(resultado)

        elif opcao == "3":
            try:
                user_id = int(input("Digite o ID do usuario: "))
                resultado = gerenciador.buscar_por_id(user_id)
                if not resultado.empty:
                    print("\nUSUARIO ENCONTRADO:")
                    resultado_display = resultado.copy()
                    resultado_display['Data_Cadastro'] = resultado_display['Data_Cadastro'].dt.strftime('%d/%m/%Y')
                    print(resultado_display.to_string(index=False))
                else:
                    print("Usuario nao encontrado.")
            except ValueError:
                print("ID deve ser um numero.")

        elif opcao == "4":
            print("\nFILTRAR USUARIOS")
            campo = input("Campo (Nome/Email/Tipo/Status/Especialidade): ")
            valor = input("Valor para filtrar: ")

            resultado = gerenciador.filtrar_usuarios(campo, valor)
            if not resultado.empty:
                resultado_display = resultado.copy()
                resultado_display['Data_Cadastro'] = resultado_display['Data_Cadastro'].dt.strftime('%d/%m/%Y')
                print(f"\nRESULTADOS PARA '{campo}' = '{valor}':")
                print(resultado_display.to_string(index=False))
            else:
                print("Nenhum usuario encontrado com esse criterio.")

        elif opcao == "5":
            try:
                user_id = int(input("ID do usuario: "))
                print(f"Status validos: {', '.join(gerenciador.status_validos)}")
                novo_status = input("Novo status: ")
                resultado = gerenciador.atualizar_status(user_id, novo_status)
                print(resultado)
            except ValueError:
                print("ID deve ser um numero.")

        elif opcao == "6":
            print("\nRELATORIOS")
            print("\nUsuarios por tipo:")
            print(gerenciador.contar_por_tipo())
            print("\nUsuarios por status:")
            print(gerenciador.contar_por_status())
            print("\nEspecialidades:")
            especialidades = gerenciador.listar_especialidades()
            if not especialidades.empty:
                print(especialidades)
            else:
                print("Nenhuma especialidade cadastrada.")

        elif opcao == "7":
            nome_arquivo = input("Nome do arquivo (pressione Enter para 'usuarios.csv'): ")
            if not nome_arquivo:
                nome_arquivo = 'usuarios.csv'
            resultado = gerenciador.exportar_csv(nome_arquivo)
            print(resultado)

        elif opcao == "8":
            gerenciador.limpar_dados()

        elif opcao == "9":
            print("Encerrando sistema...")
            break

        else:
            print("Opcao invalida.")

if __name__ == "__main__":
    menu_interativo()


SISTEMA DE USUARIOS - Verde do Amanha
1. Listar todos os usuarios
2. Adicionar novo usuario
3. Buscar usuario por ID
4. Filtrar usuarios
5. Atualizar status
6. Relatorios
7. Exportar dados
8. Limpar dados
9. Sair

Escolha uma opcao: 9
Encerrando sistema...


In [ ]:
# -*- coding: utf-8 -*-
"""Sistema de Gestão de Projetos Ambientais - Verde do Amanhã"""

import pandas as pd
from datetime import datetime, date
import os

class GestorProjetosAmbientais:
    def __init__(self):
        self.projetos = self.inicializar_dados()

    def inicializar_dados(self):
        """Inicializa a base de dados com projetos padrão"""
        return pd.DataFrame({
            'ID': [1, 2, 3],
            'Nome': ['Reflorestamento Serra Verde', 'Recuperação Solo Industrial', 'Parque Sustentável Urbano'],
            'Cliente': ['Empresa Sustentável Ltda', 'ONG EcoVida', 'Prefeitura Municipal'],
            'Tipo': ['Reflorestamento', 'Recuperação Solo', 'Cidade Verde'],
            'Area_Hectares': [50.0, 10.5, 2.3],
            'Orcamento_Real': [350000.00, 150000.00, 80000.00],
            'Status': ['Em Execução', 'Planejamento', 'Concluído'],
            'Data_Inicio': ['2024-03-01', '2024-05-15', '2024-01-10'],
            'Data_Termino': ['2024-12-31', '2024-11-30', '2024-06-30']
        })

    def limpar_console(self):
        """Limpa o console"""
        os.system('cls' if os.name == 'nt' else 'clear')

    def exibir_cabecalho(self):
        """Exibe o cabeçalho do sistema"""
        print("TABELA 2: PROJETOS AMBIENTAIS")
        print("=" * 50)

    def mostrar_todos_projetos(self):
        """Exibe todos os projetos cadastrados"""
        print("\nPROJETOS CADASTRADOS:")
        print("-" * 80)

        # Formatar valores para exibição
        df_display = self.projetos.copy()
        df_display['Orçamento (R$)'] = df_display['Orcamento_Real'].apply(lambda x: f"R$ {x:,.2f}")
        df_display['Área (ha)'] = df_display['Area_Hectares']

        # Selecionar colunas para exibição
        colunas_exibir = ['ID', 'Nome', 'Cliente', 'Tipo', 'Área (ha)', 'Orçamento (R$)', 'Status']
        print(df_display[colunas_exibir].to_string(index=False))

    def calcular_estatisticas(self):
        """Calcula e exibe estatísticas dos projetos"""
        total_orcamento = self.projetos['Orcamento_Real'].sum()
        total_area = self.projetos['Area_Hectares'].sum()
        total_projetos = len(self.projetos)

        projetos_por_status = self.projetos['Status'].value_counts()
        orcamento_medio = total_orcamento / total_projetos if total_projetos > 0 else 0

        print(f"\nESTATÍSTICAS GERAIS:")
        print("-" * 30)
        print(f"Total de projetos: {total_projetos}")
        print(f"Área total: {total_area:.1f} hectares")
        print(f"Investimento total: R$ {total_orcamento:,.2f}")
        print(f"Investimento médio: R$ {orcamento_medio:,.2f}")

        print(f"\nPROJETOS POR STATUS:")
        print("-" * 20)
        for status, quantidade in projetos_por_status.items():
            print(f"{status}: {quantidade}")

    def adicionar_projeto(self):
        """Adiciona um novo projeto"""
        print("\nADICIONAR NOVO PROJETO:")
        print("-" * 25)

        try:
            nome = input("Nome do projeto: ").strip()
            if not nome:
                print("Nome do projeto é obrigatório.")
                return False

            cliente = input("Cliente: ").strip()
            if not cliente:
                print("Cliente é obrigatório.")
                return False

            tipo = input("Tipo do projeto: ").strip()
            area = float(input("Área em hectares: "))
            orcamento = float(input("Orçamento em R$: "))

            if area <= 0 or orcamento <= 0:
                print("Área e orçamento devem ser valores positivos.")
                return False

            status = input("Status (Planejamento/Em Execução/Concluído) [Planejamento]: ").strip()
            if not status:
                status = "Planejamento"

            data_inicio = input("Data de início (YYYY-MM-DD) [hoje]: ").strip()
            if not data_inicio:
                data_inicio = datetime.now().strftime('%Y-%m-%d')

            data_termino = input("Data de término (YYYY-MM-DD) [2024-12-31]: ").strip()
            if not data_termino:
                data_termino = "2024-12-31"

            # Validar formato das datas
            try:
                datetime.strptime(data_inicio, '%Y-%m-%d')
                datetime.strptime(data_termino, '%Y-%m-%d')
            except ValueError:
                print("Formato de data inválido. Use YYYY-MM-DD")
                return False

            novo_id = self.projetos['ID'].max() + 1 if not self.projetos.empty else 1

            novo_projeto = pd.DataFrame({
                'ID': [novo_id],
                'Nome': [nome],
                'Cliente': [cliente],
                'Tipo': [tipo],
                'Area_Hectares': [area],
                'Orcamento_Real': [orcamento],
                'Status': [status],
                'Data_Inicio': [data_inicio],
                'Data_Termino': [data_termino]
            })

            self.projetos = pd.concat([self.projetos, novo_projeto], ignore_index=True)

            print("\nPROJETO ADICIONADO COM SUCESSO!")
            print("-" * 30)
            print(f"ID: {novo_id}")
            print(f"Nome: {nome}")
            print(f"Área: {area} hectares")
            print(f"Orçamento: R$ {orcamento:,.2f}")

            return True

        except ValueError:
            print("Erro: Valores numéricos inválidos.")
            return False
        except Exception as e:
            print(f"Erro inesperado: {e}")
            return False

    def filtrar_por_status(self):
        """Filtra e exibe projetos por status"""
        status_disponivel = self.projetos['Status'].unique()

        print(f"\nSTATUS DISPONÍVEIS:")
        for i, status in enumerate(status_disponivel, 1):
            print(f"{i}. {status}")

        try:
            escolha = int(input("\nEscolha o número do status: "))
            if 1 <= escolha <= len(status_disponivel):
                status_selecionado = status_disponivel[escolha - 1]
                projetos_filtrados = self.projetos[self.projetos['Status'] == status_selecionado]

                print(f"\nPROJETOS COM STATUS: {status_selecionado}")
                print("-" * 40)

                if projetos_filtrados.empty:
                    print("Nenhum projeto encontrado.")
                else:
                    df_display = projetos_filtrados.copy()
                    df_display['Orçamento (R$)'] = df_display['Orcamento_Real'].apply(lambda x: f"R$ {x:,.2f}")
                    df_display['Área (ha)'] = df_display['Area_Hectares']

                    colunas_exibir = ['ID', 'Nome', 'Cliente', 'Área (ha)', 'Orçamento (R$)']
                    print(df_display[colunas_exibir].to_string(index=False))
            else:
                print("Opção inválida.")
        except ValueError:
            print("Entrada inválida. Digite um número.")

    def calcular_impacto_ambiental(self):
        """Calcula o impacto ambiental dos projetos"""
        total_area = self.projetos['Area_Hectares'].sum()

        # Fatores de conversão estimados
        fator_co2 = 100  # toneladas CO2 por hectare/ano
        fator_oxigenio = 50  # toneladas O2 por hectare/ano
        fator_arvores = 400  # árvores por hectare (estimativa)

        co2_capturado = total_area * fator_co2
        oxigenio_gerado = total_area * fator_oxigenio
        arvores_equivalente = total_area * fator_arvores

        # Calcular por tipo de projeto
        impacto_por_tipo = self.projetos.groupby('Tipo')['Area_Hectares'].sum()

        print(f"\nIMPACTO AMBIENTAL ESTIMADO:")
        print("-" * 35)
        print(f"CO2 capturado anualmente: {co2_capturado:,.0f} toneladas")
        print(f"Oxigênio gerado anualmente: {oxigenio_gerado:,.0f} toneladas")
        print(f"Área total recuperada: {total_area:.1f} hectares")
        print(f"Equivalente a plantar: {arvores_equivalente:,.0f} árvores")

        print(f"\nIMPACTO POR TIPO DE PROJETO:")
        print("-" * 30)
        for tipo, area in impacto_por_tipo.items():
            co2_tipo = area * fator_co2
            print(f"{tipo}: {area:.1f} ha → {co2_tipo:,.0f} ton CO2/ano")

    def remover_projeto(self):
        """Remove um projeto pelo ID"""
        self.mostrar_todos_projetos()

        try:
            projeto_id = int(input("\nDigite o ID do projeto a remover: "))

            if projeto_id in self.projetos['ID'].values:
                projeto_nome = self.projetos[self.projetos['ID'] == projeto_id]['Nome'].iloc[0]
                confirmacao = input(f"Confirma remoção do projeto '{projeto_nome}'? (s/N): ").lower()

                if confirmacao == 's':
                    self.projetos = self.projetos[self.projetos['ID'] != projeto_id]
                    print("Projeto removido com sucesso!")
                else:
                    print("Remoção cancelada.")
            else:
                print("ID de projeto não encontrado.")
        except ValueError:
            print("ID inválido. Digite um número.")

    def menu_principal(self):
        """Exibe o menu principal e processa as opções"""
        while True:
            self.exibir_cabecalho()
            self.mostrar_todos_projetos()
            self.calcular_estatisticas()

            print("\n" + "=" * 50)
            print("MENU DE OPÇÕES:")
            print("1. Adicionar novo projeto")
            print("2. Filtrar projetos por status")
            print("3. Calcular impacto ambiental")
            print("4. Remover projeto")
            print("5. Limpar console")
            print("0. Sair")

            opcao = input("\nEscolha uma opção: ").strip()

            if opcao == "1":
                self.adicionar_projeto()
                input("\nPressione Enter para continuar...")

            elif opcao == "2":
                self.filtrar_por_status()
                input("\nPressione Enter para continuar...")

            elif opcao == "3":
                self.calcular_impacto_ambiental()
                input("\nPressione Enter para continuar...")

            elif opcao == "4":
                self.remover_projeto()
                input("\nPressione Enter para continuar...")

            elif opcao == "5":
                self.limpar_console()
                continue

            elif opcao == "0":
                print("Encerrando sistema...")
                break

            else:
                print("Opção inválida. Tente novamente.")
                input("Pressione Enter para continuar...")

            self.limpar_console()

def main():
    """Função principal do sistema"""
    gestor = GestorProjetosAmbientais()
    gestor.menu_principal()

if __name__ == "__main__":
    main()

TABELA 2: PROJETOS AMBIENTAIS

PROJETOS CADASTRADOS:
--------------------------------------------------------------------------------
 ID                        Nome                  Cliente             Tipo  Área (ha) Orçamento (R$)       Status
  1 Reflorestamento Serra Verde Empresa Sustentável Ltda  Reflorestamento       50.0  R$ 350,000.00  Em Execução
  2 Recuperação Solo Industrial              ONG EcoVida Recuperação Solo       10.5  R$ 150,000.00 Planejamento
  3   Parque Sustentável Urbano     Prefeitura Municipal     Cidade Verde        2.3   R$ 80,000.00    Concluído

ESTATÍSTICAS GERAIS:
------------------------------
Total de projetos: 3
Área total: 62.8 hectares
Investimento total: R$ 580,000.00
Investimento médio: R$ 193,333.33

PROJETOS POR STATUS:
--------------------
Em Execução: 1
Planejamento: 1
Concluído: 1

MENU DE OPÇÕES:
1. Adicionar novo projeto
2. Filtrar projetos por status
3. Calcular impacto ambiental
4. Remover projeto
5. Limpar console
0. Sair

Escolha u

In [ ]:
# -*- coding: utf-8 -*-
"""Sistema de Catálogo de Espécies Vegetais - Verde do Amanhã"""

import pandas as pd
import os

class CatalogoEspeciesVegetais:
    def __init__(self):
        self.especies = self.inicializar_dados()

    def inicializar_dados(self):
        """Inicializa a base de dados com espécies padrão"""
        return pd.DataFrame({
            'ID': [1, 2, 3, 4, 5],
            'Nome_Cientifico': ['Eucalyptus grandis', 'Handroanthus chrysotrichus', 'Araucaria angustifolia',
                               'Miconia cinnamomifolia', 'Schinus terebinthifolia'],
            'Nome_Popular': ['Eucalipto', 'Ipê-amarelo', 'Pinheiro-do-paraná', 'Capororoca', 'Aroeira'],
            'Tipo': ['Árvore', 'Árvore', 'Árvore', 'Arbusto', 'Árvore'],
            'Altura_Maxima': [35.0, 8.0, 25.0, 12.0, 10.0],
            'CO2_Capturado_Kg_Ano': [22.5, 15.0, 18.0, 8.5, 12.0],
            'Custo_Muda': [2.50, 8.00, 6.50, 4.00, 3.50],
            'Nativa_Brasil': ['Sim', 'Sim', 'Sim', 'Sim', 'Sim'],
            'Regiao_Ideal': ['Sul/Sudeste', 'Todo Brasil', 'Sul', 'Mata Atlântica', 'Todo Brasil'],
            'Tempo_Crescimento_Anos': [3, 5, 8, 4, 6]
        })

    def limpar_console(self):
        """Limpa o console"""
        os.system('cls' if os.name == 'nt' else 'clear')

    def exibir_cabecalho(self):
        """Exibe o cabeçalho do sistema"""
        print("TABELA 3: CATÁLOGO DE ESPÉCIES VEGETAIS")
        print("=" * 55)

    def mostrar_todas_especies(self):
        """Exibe todas as espécies cadastradas"""
        print("\nESPÉCIES CADASTRADAS:")
        print("-" * 100)

        df_display = self.especies.copy()
        df_display['Nome Científico'] = df_display['Nome_Cientifico']
        df_display['Nome Popular'] = df_display['Nome_Popular']
        df_display['Altura (m)'] = df_display['Altura_Maxima']
        df_display['CO₂ (kg/ano)'] = df_display['CO2_Capturado_Kg_Ano']
        df_display['Custo (R$)'] = df_display['Custo_Muda'].apply(lambda x: f"R$ {x:.2f}")
        df_display['Nativa'] = df_display['Nativa_Brasil']

        colunas_exibir = ['ID', 'Nome Popular', 'Nome Científico', 'Tipo',
                         'Altura (m)', 'CO₂ (kg/ano)', 'Custo (R$)', 'Nativa']
        print(df_display[colunas_exibir].to_string(index=False))

    def calcular_estatisticas(self):
        """Calcula e exibe estatísticas das espécies"""
        total_especies = len(self.especies)
        nativas = len(self.especies[self.especies['Nativa_Brasil'] == 'Sim'])
        custo_medio = self.especies['Custo_Muda'].mean()
        co2_medio = self.especies['CO2_Capturado_Kg_Ano'].mean()
        altura_media = self.especies['Altura_Maxima'].mean()

        especies_por_tipo = self.especies['Tipo'].value_counts()

        print(f"\nESTATÍSTICAS GERAIS:")
        print("-" * 25)
        print(f"Total de espécies: {total_especies}")
        print(f"Espécies nativas: {nativas} ({(nativas/total_especies)*100:.1f}%)")
        print(f"Custo médio por muda: R$ {custo_medio:.2f}")
        print(f"Captura média CO₂: {co2_medio:.1f} kg/ano")
        print(f"Altura média: {altura_media:.1f} metros")

        print(f"\nESPÉCIES POR TIPO:")
        print("-" * 18)
        for tipo, quantidade in especies_por_tipo.items():
            print(f"{tipo}: {quantidade}")

    def adicionar_especie(self):
        """Adiciona uma nova espécie"""
        print("\nADICIONAR NOVA ESPÉCIE:")
        print("-" * 23)

        try:
            nome_cientifico = input("Nome científico: ").strip()
            if not nome_cientifico:
                print("Nome científico é obrigatório.")
                return False

            nome_popular = input("Nome popular: ").strip()
            if not nome_popular:
                print("Nome popular é obrigatório.")
                return False

            print("\nTipos disponíveis: Árvore, Arbusto, Palmeira, Bambu")
            tipo = input("Tipo da planta: ").strip()
            if not tipo:
                print("Tipo é obrigatório.")
                return False

            altura_maxima = float(input("Altura máxima (metros): "))
            co2_capturado = float(input("CO₂ capturado (kg/ano): "))
            custo_muda = float(input("Custo da muda (R$): "))

            if altura_maxima <= 0 or co2_capturado < 0 or custo_muda < 0:
                print("Valores devem ser positivos.")
                return False

            nativa = input("Nativa do Brasil? (Sim/Não) [Sim]: ").strip()
            if not nativa:
                nativa = "Sim"

            regiao_ideal = input("Região ideal [Todo Brasil]: ").strip()
            if not regiao_ideal:
                regiao_ideal = "Todo Brasil"

            tempo_crescimento = input("Tempo de crescimento (anos) [5]: ").strip()
            if not tempo_crescimento:
                tempo_crescimento = 5
            else:
                tempo_crescimento = int(tempo_crescimento)

            novo_id = self.especies['ID'].max() + 1 if not self.especies.empty else 1

            nova_especie = pd.DataFrame({
                'ID': [novo_id],
                'Nome_Cientifico': [nome_cientifico],
                'Nome_Popular': [nome_popular],
                'Tipo': [tipo],
                'Altura_Maxima': [altura_maxima],
                'CO2_Capturado_Kg_Ano': [co2_capturado],
                'Custo_Muda': [custo_muda],
                'Nativa_Brasil': [nativa],
                'Regiao_Ideal': [regiao_ideal],
                'Tempo_Crescimento_Anos': [tempo_crescimento]
            })

            self.especies = pd.concat([self.especies, nova_especie], ignore_index=True)

            print("\nESPÉCIE ADICIONADA COM SUCESSO!")
            print("-" * 32)
            print(f"ID: {novo_id}")
            print(f"Nome: {nome_popular} ({nome_cientifico})")
            print(f"Tipo: {tipo}")
            print(f"Captura CO₂: {co2_capturado} kg/ano")

            return True

        except ValueError:
            print("Erro: Valores numéricos inválidos.")
            return False
        except Exception as e:
            print(f"Erro inesperado: {e}")
            return False

    def buscar_especie(self):
        """Busca espécie por nome científico ou popular"""
        busca = input("\nDigite o nome da espécie (científico ou popular): ").strip()

        if not busca:
            print("Digite um termo para busca.")
            return

        resultado = self.especies[
            self.especies['Nome_Popular'].str.contains(busca, case=False, na=False) |
            self.especies['Nome_Cientifico'].str.contains(busca, case=False, na=False)
        ]

        if len(resultado) > 0:
            print(f"\nESPÉCIES ENCONTRADAS ({len(resultado)}):")
            print("-" * 30)

            for _, especie in resultado.iterrows():
                print(f"\nID: {especie['ID']}")
                print(f"Nome Popular: {especie['Nome_Popular']}")
                print(f"Nome Científico: {especie['Nome_Cientifico']}")
                print(f"Tipo: {especie['Tipo']}")
                print(f"Altura máxima: {especie['Altura_Maxima']} metros")
                print(f"CO₂ capturado: {especie['CO2_Capturado_Kg_Ano']} kg/ano")
                print(f"Custo da muda: R$ {especie['Custo_Muda']:.2f}")
                print(f"Nativa do Brasil: {especie['Nativa_Brasil']}")
                print(f"Região ideal: {especie['Regiao_Ideal']}")
                print(f"Tempo de crescimento: {especie['Tempo_Crescimento_Anos']} anos")
                print("-" * 30)
        else:
            print(f"\nNenhuma espécie encontrada com o termo: '{busca}'")

    def filtrar_por_tipo(self):
        """Filtra espécies por tipo"""
        tipos_disponivel = self.especies['Tipo'].unique()

        print(f"\nTIPOS DISPONÍVEIS:")
        for i, tipo in enumerate(tipos_disponivel, 1):
            count = len(self.especies[self.especies['Tipo'] == tipo])
            print(f"{i}. {tipo} ({count} espécies)")

        try:
            escolha = int(input("\nEscolha o número do tipo: "))
            if 1 <= escolha <= len(tipos_disponivel):
                tipo_selecionado = tipos_disponivel[escolha - 1]
                especies_filtradas = self.especies[self.especies['Tipo'] == tipo_selecionado]

                print(f"\nESPÉCIES DO TIPO: {tipo_selecionado}")
                print("-" * 40)

                df_display = especies_filtradas.copy()
                df_display['Nome'] = df_display['Nome_Popular']
                df_display['Altura (m)'] = df_display['Altura_Maxima']
                df_display['CO₂ (kg/ano)'] = df_display['CO2_Capturado_Kg_Ano']
                df_display['Custo'] = df_display['Custo_Muda'].apply(lambda x: f"R$ {x:.2f}")

                colunas_exibir = ['ID', 'Nome', 'Altura (m)', 'CO₂ (kg/ano)', 'Custo']
                print(df_display[colunas_exibir].to_string(index=False))
            else:
                print("Opção inválida.")
        except ValueError:
            print("Entrada inválida. Digite um número.")

    def recomendar_especies(self):
        """Recomenda espécies baseado em critérios"""
        print("\nRECOMENDAÇÃO DE ESPÉCIES:")
        print("-" * 25)

        print("1. Melhores para captura de CO₂")
        print("2. Mais econômicas")
        print("3. Crescimento mais rápido")
        print("4. Apenas espécies nativas")
        print("5. Para reflorestamento (árvores grandes)")

        try:
            criterio = int(input("\nEscolha o critério: "))
            quantidade = int(input("Quantas espécies mostrar? [3]: ") or "3")

            if criterio == 1:
                recomendadas = self.especies.nlargest(quantidade, 'CO2_Capturado_Kg_Ano')
                print(f"\nTOP {quantidade} ESPÉCIES PARA CAPTURA DE CO₂:")

            elif criterio == 2:
                recomendadas = self.especies.nsmallest(quantidade, 'Custo_Muda')
                print(f"\nTOP {quantidade} ESPÉCIES MAIS ECONÔMICAS:")

            elif criterio == 3:
                recomendadas = self.especies.nsmallest(quantidade, 'Tempo_Crescimento_Anos')
                print(f"\nTOP {quantidade} ESPÉCIES DE CRESCIMENTO RÁPIDO:")

            elif criterio == 4:
                nativas = self.especies[self.especies['Nativa_Brasil'] == 'Sim']
                if len(nativas) >= quantidade:
                    recomendadas = nativas.nlargest(quantidade, 'CO2_Capturado_Kg_Ano')
                else:
                    recomendadas = nativas
                print(f"\nESPÉCIES NATIVAS RECOMENDADAS:")

            elif criterio == 5:
                arvores_grandes = self.especies[
                    (self.especies['Tipo'] == 'Árvore') &
                    (self.especies['Altura_Maxima'] >= 15)
                ]
                if len(arvores_grandes) >= quantidade:
                    recomendadas = arvores_grandes.nlargest(quantidade, 'CO2_Capturado_Kg_Ano')
                else:
                    recomendadas = arvores_grandes
                print(f"\nÁRVORES GRANDES PARA REFLORESTAMENTO:")

            else:
                print("Critério inválido.")
                return

            print("-" * 50)

            for i, (_, especie) in enumerate(recomendadas.iterrows(), 1):
                print(f"\n{i}. {especie['Nome_Popular']} ({especie['Nome_Cientifico']})")
                print(f"   CO₂: {especie['CO2_Capturado_Kg_Ano']} kg/ano")
                print(f"   Custo: R$ {especie['Custo_Muda']:.2f}")
                print(f"   Altura: {especie['Altura_Maxima']} metros")
                print(f"   Crescimento: {especie['Tempo_Crescimento_Anos']} anos")

        except ValueError:
            print("Entrada inválida. Digite números.")

    def calcular_custo_projeto(self):
        """Calcula custo de um projeto baseado em quantidade de mudas"""
        print("\nCALCULADOR DE CUSTO DE PROJETO:")
        print("-" * 32)

        self.mostrar_todas_especies()

        try:
            especies_projeto = []

            while True:
                especie_id = input("\nID da espécie (Enter para finalizar): ").strip()
                if not especie_id:
                    break

                especie_id = int(especie_id)
                if especie_id not in self.especies['ID'].values:
                    print("ID não encontrado.")
                    continue

                quantidade = int(input("Quantidade de mudas: "))
                if quantidade <= 0:
                    print("Quantidade deve ser positiva.")
                    continue

                especie = self.especies[self.especies['ID'] == especie_id].iloc[0]
                especies_projeto.append({
                    'nome': especie['Nome_Popular'],
                    'quantidade': quantidade,
                    'custo_unitario': especie['Custo_Muda'],
                    'co2_unitario': especie['CO2_Capturado_Kg_Ano'],
                    'subtotal': quantidade * especie['Custo_Muda'],
                    'co2_total': quantidade * especie['CO2_Capturado_Kg_Ano']
                })

            if especies_projeto:
                print("\nRESUMO DO PROJETO:")
                print("-" * 50)

                custo_total = 0
                co2_total = 0

                for item in especies_projeto:
                    print(f"{item['nome']}: {item['quantidade']} mudas")
                    print(f"  Custo unitário: R$ {item['custo_unitario']:.2f}")
                    print(f"  Subtotal: R$ {item['subtotal']:.2f}")
                    print(f"  CO₂ capturado: {item['co2_total']:.1f} kg/ano")
                    print()

                    custo_total += item['subtotal']
                    co2_total += item['co2_total']

                print(f"TOTAL DO PROJETO:")
                print(f"Custo total: R$ {custo_total:.2f}")
                print(f"CO₂ total capturado: {co2_total:.1f} kg/ano")
                print(f"CO₂ total capturado: {co2_total/1000:.2f} toneladas/ano")
            else:
                print("Nenhuma espécie selecionada.")

        except ValueError:
            print("Entrada inválida. Digite números.")

    def remover_especie(self):
        """Remove uma espécie pelo ID"""
        self.mostrar_todas_especies()

        try:
            especie_id = int(input("\nDigite o ID da espécie a remover: "))

            if especie_id in self.especies['ID'].values:
                especie_nome = self.especies[self.especies['ID'] == especie_id]['Nome_Popular'].iloc[0]
                confirmacao = input(f"Confirma remoção da espécie '{especie_nome}'? (s/N): ").lower()

                if confirmacao == 's':
                    self.especies = self.especies[self.especies['ID'] != especie_id]
                    print("Espécie removida com sucesso!")
                else:
                    print("Remoção cancelada.")
            else:
                print("ID de espécie não encontrado.")
        except ValueError:
            print("ID inválido. Digite um número.")

    def menu_principal(self):
        """Exibe o menu principal e processa as opções"""
        while True:
            self.exibir_cabecalho()
            self.mostrar_todas_especies()
            self.calcular_estatisticas()

            print("\n" + "=" * 55)
            print("MENU DE OPÇÕES:")
            print("1. Adicionar nova espécie")
            print("2. Buscar espécie por nome")
            print("3. Filtrar por tipo")
            print("4. Recomendar espécies")
            print("5. Calcular custo de projeto")
            print("6. Remover espécie")
            print("7. Limpar console")
            print("0. Sair")

            opcao = input("\nEscolha uma opção: ").strip()

            if opcao == "1":
                self.adicionar_especie()
                input("\nPressione Enter para continuar...")

            elif opcao == "2":
                self.buscar_especie()
                input("\nPressione Enter para continuar...")

            elif opcao == "3":
                self.filtrar_por_tipo()
                input("\nPressione Enter para continuar...")

            elif opcao == "4":
                self.recomendar_especies()
                input("\nPressione Enter para continuar...")

            elif opcao == "5":
                self.calcular_custo_projeto()
                input("\nPressione Enter para continuar...")

            elif opcao == "6":
                self.remover_especie()
                input("\nPressione Enter para continuar...")

            elif opcao == "7":
                self.limpar_console()
                continue

            elif opcao == "0":
                print("Encerrando sistema...")
                break

            else:
                print("Opção inválida. Tente novamente.")
                input("Pressione Enter para continuar...")

            self.limpar_console()

def main():
    """Função principal do sistema"""
    catalogo = CatalogoEspeciesVegetais()
    catalogo.menu_principal()

if __name__ == "__main__":
    main()

TABELA 3: CATÁLOGO DE ESPÉCIES VEGETAIS

ESPÉCIES CADASTRADAS:
----------------------------------------------------------------------------------------------------
 ID       Nome Popular            Nome Científico    Tipo  Altura (m)  CO₂ (kg/ano) Custo (R$) Nativa
  1          Eucalipto         Eucalyptus grandis  Árvore        35.0          22.5    R$ 2.50    Sim
  2        Ipê-amarelo Handroanthus chrysotrichus  Árvore         8.0          15.0    R$ 8.00    Sim
  3 Pinheiro-do-paraná     Araucaria angustifolia  Árvore        25.0          18.0    R$ 6.50    Sim
  4         Capororoca     Miconia cinnamomifolia Arbusto        12.0           8.5    R$ 4.00    Sim
  5            Aroeira    Schinus terebinthifolia  Árvore        10.0          12.0    R$ 3.50    Sim

ESTATÍSTICAS GERAIS:
-------------------------
Total de espécies: 5
Espécies nativas: 5 (100.0%)
Custo médio por muda: R$ 4.90
Captura média CO₂: 15.2 kg/ano
Altura média: 18.0 metros

ESPÉCIES POR TIPO:
------------------


In [ ]:
0# -*- coding: utf-8 -*-
"""
Sistema de Monitoramento - Verde do Amanhã
Tabela 4: Controle e acompanhamento de projetos de reflorestamento
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings

# Suprimir warnings do pandas
warnings.filterwarnings('ignore')

class MonitoramentoAmbiental:
    """Classe para gerenciar o sistema de monitoramento ambiental"""

    def __init__(self):
        """Inicializa o sistema com dados base e configurações"""
        self.dados = self._carregar_dados_iniciais()
        self.config = {
            'fator_co2_hectare': 2.25,  # Toneladas CO2 por hectare
            'sobrevivencia_minima': 70.0,  # Taxa mínima aceitável
            'densidade_mudas_hectare': 100  # Mudas por hectare padrão
        }

    def _carregar_dados_iniciais(self):
        """Carrega dados iniciais do sistema"""
        try:
            dados_base = {
                'ID': [1, 2, 3, 4, 5],
                'Projeto': ['Serra Verde', 'Serra Verde', 'Área Industrial',
                           'Parque Ecológico', 'Serra Verde'],
                'Data': ['2024-03-15', '2024-04-15', '2024-05-20',
                        '2024-06-10', '2024-07-05'],
                'Area_Monitorada_ha': [15.0, 25.0, 5.5, 12.8, 18.2],
                'Mudas_Plantadas': [1500, 2500, 600, 1280, 1820],
                'Sobrevivencia_Percent': [85.5, 92.3, 78.9, 88.1, 91.2],
                'CO2_Capturado_ton': [33.75, 56.25, 9.90, 28.80, 40.95],
                'Qualidade_Solo': ['Boa', 'Excelente', 'Regular', 'Boa', 'Excelente'],
                'Responsavel': ['João Silva', 'Maria Santos', 'Pedro Costa',
                              'Ana Oliveira', 'João Silva']
            }

            df = pd.DataFrame(dados_base)
            df['Data'] = pd.to_datetime(df['Data'])
            return df

        except Exception as e:
            print(f"Erro ao carregar dados iniciais: {e}")
            return pd.DataFrame()

    def validar_dados(self, **kwargs):
        """Valida dados de entrada para novo registro"""
        erros = []

        # Validações obrigatórias
        if not kwargs.get('projeto') or len(kwargs['projeto'].strip()) == 0:
            erros.append("Nome do projeto é obrigatório")

        if kwargs.get('area', 0) <= 0:
            erros.append("Área deve ser maior que zero")

        if kwargs.get('mudas', 0) < 0:
            erros.append("Número de mudas não pode ser negativo")

        if not (0 <= kwargs.get('sobrevivencia', -1) <= 100):
            erros.append("Taxa de sobrevivência deve estar entre 0 e 100%")

        qualidade_valida = ['Excelente', 'Boa', 'Regular', 'Ruim']
        if kwargs.get('qualidade_solo') not in qualidade_valida:
            erros.append(f"Qualidade do solo deve ser uma das opções: {qualidade_valida}")

        return erros

    def adicionar_registro(self, projeto, area, mudas, sobrevivencia,
                          qualidade_solo='Regular', responsavel='N/A'):
        """Adiciona novo registro de monitoramento com validação"""

        # Validar dados
        erros = self.validar_dados(
            projeto=projeto, area=area, mudas=mudas,
            sobrevivencia=sobrevivencia, qualidade_solo=qualidade_solo
        )

        if erros:
            return False, f"Erros de validação: {'; '.join(erros)}"

        try:
            # Calcular CO2 capturado
            co2_capturado = area * self.config['fator_co2_hectare']

            # Criar novo registro
            novo_id = self.dados['ID'].max() + 1 if not self.dados.empty else 1

            novo_registro = {
                'ID': novo_id,
                'Projeto': projeto.strip(),
                'Data': datetime.now(),
                'Area_Monitorada_ha': float(area),
                'Mudas_Plantadas': int(mudas),
                'Sobrevivencia_Percent': float(sobrevivencia),
                'CO2_Capturado_ton': round(co2_capturado, 2),
                'Qualidade_Solo': qualidade_solo,
                'Responsavel': responsavel.strip()
            }

            # Adicionar ao DataFrame
            self.dados = pd.concat([self.dados, pd.DataFrame([novo_registro])],
                                 ignore_index=True)

            return True, f"Registro {novo_id} adicionado com sucesso"

        except Exception as e:
            return False, f"Erro ao adicionar registro: {e}"

    def obter_metricas_gerais(self):
        """Calcula métricas consolidadas do sistema"""
        if self.dados.empty:
            return {}

        try:
            metricas = {
                'total_registros': len(self.dados),
                'total_mudas': int(self.dados['Mudas_Plantadas'].sum()),
                'area_total': float(self.dados['Area_Monitorada_ha'].sum()),
                'co2_total': float(self.dados['CO2_Capturado_ton'].sum()),
                'sobrevivencia_media': float(self.dados['Sobrevivencia_Percent'].mean()),
                'projetos_ativos': self.dados['Projeto'].nunique(),
                'periodo_dados': {
                    'inicio': self.dados['Data'].min().strftime('%Y-%m-%d'),
                    'fim': self.dados['Data'].max().strftime('%Y-%m-%d')
                }
            }

            # Métricas por qualidade do solo
            metricas['distribuicao_qualidade'] = self.dados['Qualidade_Solo'].value_counts().to_dict()

            # Alertas
            metricas['alertas'] = self._gerar_alertas()

            return metricas

        except Exception as e:
            print(f"Erro ao calcular métricas: {e}")
            return {}

    def _gerar_alertas(self):
        """Gera alertas baseados nos dados de monitoramento"""
        alertas = []

        try:
            # Alerta para baixa sobrevivência
            baixa_sobrevivencia = self.dados[
                self.dados['Sobrevivencia_Percent'] < self.config['sobrevivencia_minima']
            ]

            if not baixa_sobrevivencia.empty:
                alertas.append(f"ATENÇÃO: {len(baixa_sobrevivencia)} registro(s) com "
                             f"taxa de sobrevivência abaixo de {self.config['sobrevivencia_minima']}%")

            # Alerta para qualidade do solo ruim
            solo_ruim = self.dados[self.dados['Qualidade_Solo'] == 'Ruim']
            if not solo_ruim.empty:
                alertas.append(f"ATENÇÃO: {len(solo_ruim)} área(s) com qualidade de solo ruim")

            # Alerta para projetos sem monitoramento recente
            data_limite = datetime.now() - timedelta(days=60)
            projetos_antigos = self.dados.groupby('Projeto')['Data'].max()
            projetos_desatualizados = projetos_antigos[projetos_antigos < data_limite]

            if not projetos_desatualizados.empty:
                alertas.append(f"ATENÇÃO: {len(projetos_desatualizados)} projeto(s) sem "
                             "monitoramento nos últimos 60 dias")

        except Exception as e:
            alertas.append(f"Erro ao gerar alertas: {e}")

        return alertas

    def filtrar_dados(self, projeto=None, data_inicio=None, data_fim=None,
                     sobrevivencia_min=None):
        """Filtra dados baseado em critérios específicos"""
        try:
            dados_filtrados = self.dados.copy()

            if projeto:
                dados_filtrados = dados_filtrados[
                    dados_filtrados['Projeto'].str.contains(projeto, case=False, na=False)
                ]

            if data_inicio:
                data_inicio = pd.to_datetime(data_inicio)
                dados_filtrados = dados_filtrados[dados_filtrados['Data'] >= data_inicio]

            if data_fim:
                data_fim = pd.to_datetime(data_fim)
                dados_filtrados = dados_filtrados[dados_filtrados['Data'] <= data_fim]

            if sobrevivencia_min:
                dados_filtrados = dados_filtrados[
                    dados_filtrados['Sobrevivencia_Percent'] >= sobrevivencia_min
                ]

            return dados_filtrados

        except Exception as e:
            print(f"Erro ao filtrar dados: {e}")
            return pd.DataFrame()

    def gerar_relatorio_projeto(self, nome_projeto):
        """Gera relatório detalhado para um projeto específico"""
        try:
            dados_projeto = self.dados[self.dados['Projeto'] == nome_projeto]

            if dados_projeto.empty:
                return f"Nenhum dado encontrado para o projeto '{nome_projeto}'"

            relatorio = {
                'projeto': nome_projeto,
                'total_monitoramentos': len(dados_projeto),
                'area_total': float(dados_projeto['Area_Monitorada_ha'].sum()),
                'mudas_totais': int(dados_projeto['Mudas_Plantadas'].sum()),
                'co2_total': float(dados_projeto['CO2_Capturado_ton'].sum()),
                'sobrevivencia_media': float(dados_projeto['Sobrevivencia_Percent'].mean()),
                'ultimo_monitoramento': dados_projeto['Data'].max().strftime('%Y-%m-%d'),
                'responsaveis': dados_projeto['Responsavel'].unique().tolist(),
                'evolucao_sobrevivencia': dados_projeto.sort_values('Data')[
                    ['Data', 'Sobrevivencia_Percent']
                ].to_dict('records')
            }

            return relatorio

        except Exception as e:
            return f"Erro ao gerar relatório: {e}"

    def exportar_dados(self, formato='csv', caminho=None):
        """Exporta dados para arquivo"""
        try:
            if caminho is None:
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                caminho = f'monitoramento_ambiental_{timestamp}'

            if formato.lower() == 'csv':
                arquivo = f'{caminho}.csv'
                self.dados.to_csv(arquivo, index=False, encoding='utf-8')
                return f"Dados exportados para: {arquivo}"

            elif formato.lower() == 'excel':
                arquivo = f'{caminho}.xlsx'
                self.dados.to_excel(arquivo, index=False, engine='openpyxl')
                return f"Dados exportados para: {arquivo}"

            else:
                return "Formato não suportado. Use 'csv' ou 'excel'"

        except Exception as e:
            return f"Erro ao exportar dados: {e}"

    def obter_dados_formatados(self):
        """Retorna dados formatados para exibição"""
        try:
            if self.dados.empty:
                return "Nenhum dado disponível"

            # Formatação para exibição
            dados_exibicao = self.dados.copy()
            dados_exibicao['Data'] = dados_exibicao['Data'].dt.strftime('%Y-%m-%d')
            dados_exibicao['Area_Monitorada_ha'] = dados_exibicao['Area_Monitorada_ha'].round(1)
            dados_exibicao['Sobrevivencia_Percent'] = dados_exibicao['Sobrevivencia_Percent'].round(1)
            dados_exibicao['CO2_Capturado_ton'] = dados_exibicao['CO2_Capturado_ton'].round(2)

            return dados_exibicao

        except Exception as e:
            return f"Erro ao formatar dados: {e}"

def menu_interativo(sistema):
    """Menu interativo para operações do sistema"""
    while True:
        print("\n" + "=" * 60)
        print("MENU DE OPÇÕES:")
        print("1. Visualizar dados de monitoramento")
        print("2. Adicionar novo registro")
        print("3. Filtrar dados")
        print("4. Relatório por projeto")
        print("5. Métricas consolidadas")
        print("6. Exportar dados")
        print("7. Sair")
        print("-" * 60)

        try:
            opcao = input("Escolha uma opção (1-7): ").strip()

            if opcao == '1':
                visualizar_dados(sistema)
            elif opcao == '2':
                adicionar_novo_registro(sistema)
            elif opcao == '3':
                filtrar_dados_interativo(sistema)
            elif opcao == '4':
                relatorio_projeto_interativo(sistema)
            elif opcao == '5':
                exibir_metricas(sistema)
            elif opcao == '6':
                exportar_dados_interativo(sistema)
            elif opcao == '7':
                print("Encerrando sistema...")
                break
            else:
                print("Opção inválida! Tente novamente.")

        except KeyboardInterrupt:
            print("\n\nSistema encerrado pelo usuário.")
            break
        except Exception as e:
            print(f"Erro na operação: {e}")

def visualizar_dados(sistema):
    """Exibe dados de monitoramento formatados"""
    print("\nDADOS DE MONITORAMENTO ATUAL:")
    print("-" * 80)
    dados_formatados = sistema.obter_dados_formatados()
    if isinstance(dados_formatados, str):
        print(dados_formatados)
    else:
        print(dados_formatados.to_string(index=False))

def adicionar_novo_registro(sistema):
    """Interface para adicionar novo registro"""
    print("\nADICIONAR NOVO REGISTRO DE MONITORAMENTO:")
    print("-" * 50)

    try:
        projeto = input("Nome do projeto: ").strip()
        if not projeto:
            print("Nome do projeto é obrigatório!")
            return

        area = input("Área monitorada (hectares): ").strip()
        try:
            area = float(area)
            if area <= 0:
                print("Área deve ser maior que zero!")
                return
        except ValueError:
            print("Área deve ser um número válido!")
            return

        mudas = input("Número de mudas plantadas: ").strip()
        try:
            mudas = int(mudas)
            if mudas < 0:
                print("Número de mudas não pode ser negativo!")
                return
        except ValueError:
            print("Número de mudas deve ser um número inteiro válido!")
            return

        sobrevivencia = input("Taxa de sobrevivência (%): ").strip()
        try:
            sobrevivencia = float(sobrevivencia)
            if not (0 <= sobrevivencia <= 100):
                print("Taxa de sobrevivência deve estar entre 0 e 100%!")
                return
        except ValueError:
            print("Taxa de sobrevivência deve ser um número válido!")
            return

        print("\nOpções de qualidade do solo:")
        print("1. Excelente")
        print("2. Boa")
        print("3. Regular")
        print("4. Ruim")

        opcao_qualidade = input("Selecione qualidade do solo (1-4): ").strip()
        qualidades = {'1': 'Excelente', '2': 'Boa', '3': 'Regular', '4': 'Ruim'}
        qualidade_solo = qualidades.get(opcao_qualidade, 'Regular')

        responsavel = input("Nome do responsável (opcional): ").strip()
        if not responsavel:
            responsavel = 'N/A'

        # Tentar adicionar registro
        sucesso, mensagem = sistema.adicionar_registro(
            projeto=projeto,
            area=area,
            mudas=mudas,
            sobrevivencia=sobrevivencia,
            qualidade_solo=qualidade_solo,
            responsavel=responsavel
        )

        print(f"\n{mensagem}")

        if sucesso:
            print("\nDados do novo registro:")
            ultimo_registro = sistema.dados.tail(1)
            print(ultimo_registro.to_string(index=False))

    except Exception as e:
        print(f"Erro ao adicionar registro: {e}")

def filtrar_dados_interativo(sistema):
    """Interface para filtrar dados"""
    print("\nFILTRAR DADOS DE MONITORAMENTO:")
    print("-" * 40)

    try:
        projeto = input("Nome do projeto (deixe vazio para todos): ").strip()
        projeto = projeto if projeto else None

        data_inicio = input("Data início (YYYY-MM-DD, deixe vazio para ignorar): ").strip()
        data_inicio = data_inicio if data_inicio else None

        data_fim = input("Data fim (YYYY-MM-DD, deixe vazio para ignorar): ").strip()
        data_fim = data_fim if data_fim else None

        sobrevivencia_min = input("Taxa mínima de sobrevivência (deixe vazio para ignorar): ").strip()
        if sobrevivencia_min:
            try:
                sobrevivencia_min = float(sobrevivencia_min)
            except ValueError:
                print("Taxa de sobrevivência deve ser um número válido!")
                sobrevivencia_min = None
        else:
            sobrevivencia_min = None

        dados_filtrados = sistema.filtrar_dados(
            projeto=projeto,
            data_inicio=data_inicio,
            data_fim=data_fim,
            sobrevivencia_min=sobrevivencia_min
        )




        if dados_filtrados.empty:
            print("\nNenhum registro encontrado com os filtros aplicados.")
        else:
            print(f"\nDADOS FILTRADOS ({len(dados_filtrados)} registro(s)):")
            print("-" * 80)
            dados_formatados = dados_filtrados.copy()
            dados_formatados['Data'] = dados_formatados['Data'].dt.strftime('%Y-%m-%d')
            print(dados_formatados.to_string(index=False))

    except Exception as e:
        print(f"Erro ao filtrar dados: {e}")

def relatorio_projeto_interativo(sistema):
    """Interface para relatório por projeto"""
    print("\nRELATÓRIO POR PROJETO:")
    print("-" * 30)

    try:
        # Mostrar projetos disponíveis
        projetos = sistema.dados['Projeto'].unique()
        print("Projetos disponíveis:")
        for i, projeto in enumerate(projetos, 1):
            print(f"{i}. {projeto}")

        nome_projeto = input("\nDigite o nome do projeto: ").strip()

        if nome_projeto in projetos:
            relatorio = sistema.gerar_relatorio_projeto(nome_projeto)

            if isinstance(relatorio, dict):
                print(f"\nRELATÓRIO DO PROJETO: {relatorio['projeto']}")
                print("-" * 50)
                print(f"Total de monitoramentos: {relatorio['total_monitoramentos']}")
                print(f"Área total: {relatorio['area_total']:.1f} ha")
                print(f"Mudas totais: {relatorio['mudas_totais']:,}")
                print(f"CO2 total capturado: {relatorio['co2_total']:.2f} toneladas")
                print(f"Taxa média sobrevivência: {relatorio['sobrevivencia_media']:.1f}%")
                print(f"Último monitoramento: {relatorio['ultimo_monitoramento']}")
                print(f"Responsáveis: {', '.join(relatorio['responsaveis'])}")
            else:
                print(relatorio)
        else:
            print("Projeto não encontrado!")

    except Exception as e:
        print(f"Erro ao gerar relatório: {e}")

def exibir_metricas(sistema):
    """Exibe métricas consolidadas"""
    print("\nMÉTRICAS CONSOLIDADAS:")
    print("-" * 40)

    try:
        metricas = sistema.obter_metricas_gerais()

        if metricas:
            print(f"Total de registros: {metricas.get('total_registros', 0)}")
            print(f"Mudas plantadas: {metricas.get('total_mudas', 0):,}")
            print(f"Área total monitorada: {metricas.get('area_total', 0):.1f} ha")
            print(f"CO2 capturado: {metricas.get('co2_total', 0):.2f} toneladas")
            print(f"Taxa média sobrevivência: {metricas.get('sobrevivencia_media', 0):.1f}%")
            print(f"Projetos ativos: {metricas.get('projetos_ativos', 0)}")

            # Distribuição por qualidade do solo
            if metricas.get('distribuicao_qualidade'):
                print("\nDistribuição por qualidade do solo:")
                for qualidade, quantidade in metricas['distribuicao_qualidade'].items():
                    print(f"  {qualidade}: {quantidade}")

            # Exibir alertas
            if metricas.get('alertas'):
                print(f"\nALERTAS:")
                for alerta in metricas['alertas']:
                    print(f"• {alerta}")
        else:
            print("Não foi possível calcular métricas.")

    except Exception as e:
        print(f"Erro ao exibir métricas: {e}")

def exportar_dados_interativo(sistema):
    """Interface para exportar dados"""
    print("\nEXPORTAR DADOS:")
    print("-" * 20)

    try:
        print("Formatos disponíveis:")
        print("1. CSV")
        print("2. Excel")

        formato_opcao = input("Escolha o formato (1-2): ").strip()
        formatos = {'1': 'csv', '2': 'excel'}
        formato = formatos.get(formato_opcao, 'csv')

        nome_arquivo = input("Nome do arquivo (sem extensão): ").strip()
        if not nome_arquivo:
            nome_arquivo = None

        resultado = sistema.exportar_dados(formato=formato, caminho=nome_arquivo)
        print(f"\n{resultado}")

    except Exception as e:
        print(f"Erro ao exportar dados: {e}")

def main():
    """Função principal com menu interativo"""
    print("SISTEMA DE MONITORAMENTO AMBIENTAL - VERDE DO AMANHÃ")
    print("=" * 60)

    try:
        # Inicializar sistema
        sistema = MonitoramentoAmbiental()
        print("Sistema inicializado com sucesso!")

        # Exibir resumo inicial
        metricas = sistema.obter_metricas_gerais()
        if metricas:
            print(f"\nResumo: {metricas.get('total_registros', 0)} registros, "
                  f"{metricas.get('projetos_ativos', 0)} projetos ativos")

        # Iniciar menu interativo
        menu_interativo(sistema)

        return sistema

    except Exception as e:
        print(f"Erro ao inicializar sistema: {e}")
        return None

if __name__ == "__main__":
    sistema_monitoramento = main()

SISTEMA DE MONITORAMENTO AMBIENTAL - VERDE DO AMANHÃ
Sistema inicializado com sucesso!

Resumo: 5 registros, 3 projetos ativos

MENU DE OPÇÕES:
1. Visualizar dados de monitoramento
2. Adicionar novo registro
3. Filtrar dados
4. Relatório por projeto
5. Métricas consolidadas
6. Exportar dados
7. Sair
------------------------------------------------------------
Escolha uma opção (1-7): 7
Encerrando sistema...
